# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio


  Obtaining dependency information for fiftyone==0.21.4 from https://files.pythonhosted.org/packages/0e/1f/062e3087fa672a8324c969521670e5475edecb8005ab6c9f872a274db784/fiftyone-0.21.4-py3-none-any.whl.metadata
  Obtaining dependency information for argcomplete from https://files.pythonhosted.org/packages/1e/05/223116a4a5905d6b26bff334ffc7b74474fafe23fcb10532caf0ef86ca69/argcomplete-3.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/9b/86/1c5a07d3c93e80b22e6dce8c2280199e6e12dbf72dd8738d4f5e65887bc8/boto3-1.28.56-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools from https://files.pythonhosted.org/packages/a9/c9/c8a7710f2cedcb1db9224fdd4d8307c9e48cbddc46c18b515fefc0f1abbe/cachetools-5.3.1-py3-none-any.whl.metadata
  Using cached cachetools-5.3.1-py3-none-any.whl.metadata (5.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 153.1 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Using cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 518.2 kB/s eta 0:00:00m eta 0:00:010:00:02
  Obtaining dependency information for voxel51-eta<0.11,>=0.10 from https://files.pythonhosted.org/packages/97/d1/8fb3e3c96c55a19b5498d6acc7437b12250fb092edaa9f8c16f38836897f/voxel51_eta-0.10.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/21/6d/abf701fa71ff22e3617ec9b46197f9ff5bba16dfefa7ee259b60216112eb/opencv_python_headless-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 908.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 804.3 kB/s eta 0:00:00MB/s eta 0:00:01
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e45421

  Obtaining dependency information for tifffile>=2022.8.12 from https://files.pythonhosted.org/packages/06/a3/68d17088a4f09565bc7341fd20490da8191ec4cddde479daaabbe07bb603/tifffile-2023.7.10-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 978.7 kB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for lazy_loader>=0.2 from https://files.pythonhosted.org/packages/a1/c3/65b3814e155836acacf720e5be3b5757130346670ac454fee29d3eda1381/lazy_loader-0.3-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore<0.19.0,>=0.18.0 from https://files.pythonhosted.org/packages/ac/97/724afbb7925339f6214bf1fdb5714d1a462690466832bf8fb3fd497649f1/httpcore-0.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for pycryptodomex>=3.6.6 from https://files.pythonhosted.org/packages/23/23/3f3d042c96ff7bece5b126365593b1f9c8e3ae62ce80d44e9da39c5e8a73/pycryptodomex-3.19.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metada

Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os



Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [ ]:
dataset = fo.Dataset.from_images_dir('D:\\AIC\\KeyFrames', name=None, tags=None, recursive=True)

Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [ ]:
session = fo.launch_app(dataset, auto=False)

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [ ]:
session.open_tab()

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [ ]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [ ]:
print(dataset.first())

### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
for sample in dataset:
    object_path = f"D:\\AIC\\objects\\{sample['video']}\\{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()


### Thêm thông tin CLIP embedding.

In [ ]:
all_keyframe = glob('D:\\AIC\\Keyframes\\*\\keyframes\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('D:\\AIC\\Keyframes\\*\\keyframes\\*')
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = f'D:\\AIC\\clip-features-vit-b32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [ ]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
# Bạn cần phải cài version umap-learn hỗ trợ.
# fob.compute_visualization(
#     dataset,
#     embeddings=clip_embeddings,
#     brain_key="img_viz"
# )
